In [ ]:
import pymongo
from splinter import Browser
from bs4 import BeautifulSoup
import os
import pandas as pd

#input desired NYC neighborhood with spaces separated by hyphen <example> 'upper-west-side'

def get_pull_page(n):
    def init_browser():
        import platform
        if platform.system().lower() == 'windows'.lower():
            executable_path = {
                'executable_path': 
                os.path.join(os.getcwd(), 'chromedriver.exe')}
            return Browser('chrome', **executable_path, headless=False)
        else:
            return Browser('chrome')

    def get_html(browser, url = 'https://www.zillow.com/homes/for_rent/' + n):
        browser.visit(url)
        html = browser.html
        return html
    
    
    def get_listings(html):
        soup = BeautifulSoup(html, "html.parser")
        address = soup.find_all('h3', class_= 'list-card-addr')
        price = soup.find_all('div', class_= 'list-card-price')
        #size = soup.find_all('div', class_= 'list-card-price')
        return address, price
    
    def scrape(browser):
        html = get_html(browser)
        address_list, price_list = get_listings(html)
        return address_list, price_list
    
    def access_db(zillowscrape, For_Rent):
        conn = 'mongodb://localhost:27017'
        client = pymongo.MongoClient(conn)
        db = client['zillowscrape']
        collection = db['For_Rent']
        return db, collection
    
    def main(n): # put n here
        db, collection = access_db('zillowscrape', 'For_Rent')
        browser = init_browser()
        address_titles, price_titles=scrape(browser)
        address_join = ','.join([t.text for t in address_titles])
        price_join = ','.join([t.text for t in price_titles])
        return address_join, price_join   
    listings = main(n) # build n into main() func, pass a city or some variable into the main function
    return listings
search = get_pull_page('upper-west-side')
search

In [23]:
city_search = {'City':search}

In [24]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["zillowscrape"]
mycol = mydb["For_Rent"]
mydict = mycol.insert_one(city_search)